In [ ]:
!pip install torch lightning numpy kaggle wandb torch-geometric polars


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.7/777.7 kB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.7 MB/s eta 0:00:00


In [ ]:
!pip install polars -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.4/28.4 MB 36.2 MB/s eta 0:00:00
  Attempting uninstall: polars
    Found existing installation: polars 0.17.3
    Uninstalling polars-0.17.3:
      Successfully uninstalled polars-0.17.3


In [ ]:
from google.colab import files

# Carica il file kaggle.json
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"alexxxyy47","key":"2c3d331934a8a1d64227f77606e63d4c"}'}

In [ ]:
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d ealaxi/paysim1
!unzip paysim1.zip
!rm paysim1.zip

 96% 171M/178M [00:01<00:00, 136MB/s]
100% 178M/178M [00:01<00:00, 144MB/s]
Archive:  paysim1.zip
  inflating: PS_20174392719_1491204439457_log.csv  


In [ ]:
import pandas as pd, sys, plotly.graph_objects as go, plotly.express as px, numpy as np, torch, random as rnd, torch.nn as nn, lightning as L, wandb as wndb, pdb
from torch.utils.data import Dataset, DataLoader
from sklearn.utils import shuffle
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx
from matplotlib import pyplot as plt
import networkx as nx
from torch_geometric.nn import GCNConv
import torch_geometric.transforms as T
from torch import optim
from torchmetrics.classification import BinaryF1Score
from torch_geometric.nn import MessagePassing
from typing import Optional
from torch.nn import Parameter
from torch_geometric.typing import (
    Adj,
    OptPairTensor,
    OptTensor,
    SparseTensor,
    torch_sparse,
)
from torch_geometric.utils import add_remaining_self_loops
from torch_geometric.utils import add_self_loops as add_self_loops_fn
from torch_geometric.utils import (
    is_torch_sparse_tensor,
    scatter,
    spmm,
    to_edge_index,
)
from torch_geometric.utils.num_nodes import maybe_num_nodes
from torch_geometric.utils.sparse import set_sparse_value
from torch_geometric.nn.dense.linear import Linear
from torch_geometric.nn.inits import zeros
import polars as pl, json
from torch_geometric.nn. import Sequential




In [ ]:
# PARAMETERS

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
ACCELERATOR =  "gpu" if torch.cuda.is_available() else "cpu"
SEED = 42

rnd.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

DEPTH = 2
LR = 1e-3
WD = 1e-5
H_SIZE = 8

"aaf831dabc88d936d4e6b439b798bb4cb42814ea"

#wndb.login()



'aaf831dabc88d936d4e6b439b798bb4cb42814ea'

In [ ]:
# UTILS FUNCTIONS

def load_dataframe( dataset_file : str):
    return pl.read_csv(dataset_file)


def find_null_or_empty_records( dataframe):
    return # dataframe.filter(dataframe.apply(lambda col: col.is_nan().any()))

def print_progress_bar(percentuale, lunghezza_barra=20):
    blocchi_compilati = int(lunghezza_barra * percentuale)
    barra = "[" + "=" * (blocchi_compilati - 1) + ">" + " " * (lunghezza_barra - blocchi_compilati) + "]"
    sys.stdout.write(f"\r{barra} {percentuale * 100:.2f}% completo")
    sys.stdout.flush()


def compute_kind_inconsistence(dataframe):
    return {"inconsistent orig balance": len(dataframe.filter(abs(pl.col('oldbalanceOrg') - pl.col('newbalanceOrig')) != pl.col('amount')))/len(dataframe),
            "inconsistent dest balance": len(dataframe.filter(abs(pl.col('oldbalanceDest') - pl.col('newbalanceDest')) != pl.col('amount')))/len(dataframe),
            "zero cash transaction": len(dataframe.filter(pl.col('amount') == 0 ))/len(dataframe),
            "self-transaction": len(dataframe.filter(pl.col('nameOrig') == pl.col('nameDest')))/len(dataframe)
            }

def plot_histogram(to_plot):


    # Converti il dizionario in un array di valori
    values = list(to_plot.values())

    # Crea un istogramma
    fig = go.Figure(data=[go.Bar(x=list(to_plot.keys()), y=values)])

    # Mostra l'istogramma
    fig.show()



def plot_categories(dataframe):
    # Calcola la frequenza di ogni categoria nella colonna 'type'
    counts = dataframe.groupby('type').count()

    # Crea l'istogramma con Plotly Express
    fig = px.bar(counts, x='type', y='count', title='Istogramma delle categorie nella colonna "type"')

    # Mostra il plot
    fig.show()


def build_names_conversion_index(df):
  all_names = pl.DataFrame({"names": pl.concat([df['nameDest'] , df['nameOrig']])}).select("names").unique()
  # Costruisci il dizionario di mapping
  all_names = pl.DataFrame({"names": all_names["names"], "id": [i for i in range(len(all_names["names"]))]})

  try:
    with open("name_to_id.json","r") as f:
      return json.load(f), all_names
  except:
    name_to_id = dict(zip(all_names['names'], all_names['id']))
    with open("name_to_id.json","w") as f:
      json.dump(name_to_id, f)
      #breakpoint()
      return name_to_id , all_names


def compute_mean_var(df, name,var=False):
  try:

    orig = df.filter(pl.col('nameOrig') == name).select(pl.col('newbalanceOrig'))['newbalanceOrig']
    dest = df.filter(pl.col('nameDest') == name).select(pl.col('newbalanceDest'))['newbalanceDest']
    if name == 'C1831599196':
      breakpoint()
    if var:
      return dest.var() if orig.shape[0] == 0 else orig.var() if dest.shape[0] == 0 else (len(orig) * orig.var() + len(dest) * dest.var()) / (len(orig) + len(dest))

    else:
      return dest.mean() if orig.shape[0] == 0 else orig.mean() if dest.shape[0] == 0 else (orig.mean() + dest.mean()) / 2
  except:
    print(name)


def combine_lists(l1,l2):
  l3 = []
  for el in l1:
    if not el is None:
      l3.append(el)
  for el in l2:
    if not el is None:
      l3.append(el)
  return l3

def str_to_list(s):
  l = s.split(',')
  return [float(e) for e in l]


def random_split(df, train_prc, val_prc):
  df_ids = df.with_columns(pl.col("id",list(range(len(df)))))
  pos = df_ids.filter(pl.col('isFraud') == 1).select(pl.col("id"))
  neg = df_ids.filter(pl.col('isFraud') == 0).select(pl.col("id"))
  val_len_pos = len(pos) * val_prc
  val_len_neg = len(neg) * val_prc
  pos_train_ids = random.sample(pos['id'], len(pos) * train_prc)
  pos_train = pos.filter(pl.col('id').isin(pos_train_ids))
  pos = pos.filter(~pl.col('id').isin(pos_train_ids))

  neg_train_ids = random.sample(neg['id'], len(neg) * train_prc)
  neg_train = neg.filter(pl.col('id').isin(neg_train_ids))
  neg = neg.filter(~pl.col('id').isin(neg_train_ids))

  pos_val_ids = random.sample(pos['id'], val_len_pos)
  pos_val = pos.filter(pl.col('id').isin(pos_val_ids))
  pos = pos.filter(~pl.col('id').isin(pos_val_ids))

  neg_val_ids = random.sample(neg['id'], val_len_neg)
  neg_val = neg.filter(pl.col('id').isin(neg_val_ids))
  neg = neg.filter(~pl.col('id').isin(neg_val_ids))

  train_set = pl.concat([pos_train, neg_train])
  val_set = pl.concat([pos_val, neg_val])
  test_set = pl.concat([pos, neg])
  return train_set, val_set, test_set













In [ ]:
class FraudDetectionDataset(Dataset):

    def __init__(self,dataset_file : str):
        dataframe = load_dataframe(dataset_file)

        dataframe = dataframe.with_columns(pl.col("isFraud").cast(int).alias("isFraud"))
        self.raw_data = dataframe
        self.transaction_types = {
            "CASH_IN": 0,
            "CASH_OUT": 1,
            "DEBIT": 2,
            "PAYMENT": 3,
            "TRANSFER": 4
        }

    def analize_data(self,find_empty_records = True):
        print("----HEAD----")
        print(self.raw_data.head())
        print("----DESCRIBE----")
        print(self.raw_data.describe())
        if find_empty_records:
          find_null_or_empty_records(self.raw_data)

    def extract_inconsistent_transactions(self):

        return self.raw_data.filter(
            (abs(pl.col('oldbalanceOrg') - pl.col('newbalanceOrig')) != pl.col('amount')) |
            (abs(pl.col('oldbalanceDest') - pl.col('newbalanceDest')) != pl.col('amount')) | (pl.col('amount') == 0) | (pl.col('nameOrig') == pl.col('nameDest'))
        )


    def remove_inconsistent(self):
        self.raw_data = self.raw_data.filter(((abs(pl.col('oldbalanceOrg') - pl.col('newbalanceOrig')) == abs(pl.col('oldbalanceDest') - pl.col('newbalanceDest'))) & ~pl.col("nameDest").startswith("M")) & (pl.col('amount') != 0))


    def convert_type(self):
      self.raw_data = self.raw_data.select(pl.exclude("isFlaggedFraud"))
      self.raw_data = self.raw_data.with_columns(pl.col("type").map_elements(lambda x: self.transaction_types[x]).alias("type"))



    def build_graph(self, train_prc, val_prc):
      self.train_edges, self.val_edges, self.test_edges = random_split(self.raw_data, train_prc, val_prc)


      node_names, x_dataframe = build_names_conversion_index(self.raw_data)
      #breakpoint()/
      #breakpoint()
      join1 = x_dataframe.join(self.raw_data.select(pl.col("nameOrig",'oldbalanceOrg','amount')), left_on='names', right_on='nameOrig')
      join2 = join1.join(self.raw_data.select(pl.col("nameDest",'oldbalanceDest'),pl.col('amount').alias('amount2')), left_on='names', right_on='nameDest')
      group1 = join2.lazy().group_by("names").agg(pl.col("oldbalanceOrg").alias("grouped_balanceOrig"), pl.col('amount').alias('grouped_amount'),
            pl.col('oldbalanceDest').alias('grouped_balanceDest'), pl.col('amount2').alias('grouped_amount2') )
      group2 = group1.collect().with_columns(pl.col('grouped_balanceDest').map_elements(lambda x: str(list(x))[1:-1]).alias('grouped_balanceDest'),
                                             pl.col('grouped_balanceOrig').map_elements(lambda x: str(list(x))[1:-1]).alias('grouped_balanceOrig'),
                                             pl.col('grouped_amount').map_elements(lambda x: str(list(x))[1:-1]).alias('grouped_amount'),
                                             pl.col('grouped_amount2').map_elements(lambda x: str(list(x))[1:-1]).alias('grouped_amount2'))
      group3 = group2.select(
          pl.col('names'),
          pl.reduce(
            lambda x, y:
             x + "," + y, exprs=pl.col("grouped_balanceOrig","grouped_balanceDest")
        ).alias("balance"),
          pl.reduce(
            lambda x, y:
             x + "," + y, exprs=pl.col("grouped_amount","grouped_amount2")
        ).alias("amount")
        )
      group4 = group3.with_columns(
          pl.col('balance').map_elements(lambda x : np.mean(str_to_list(x))).alias('mean_balance'),
          pl.col('balance').map_elements(lambda x : np.var(str_to_list(x))).alias('variance_balance'),
          pl.col('amount').map_elements(lambda x : np.mean(str_to_list(x))).alias('mean_amount'),
          pl.col('amount').map_elements(lambda x : np.var(str_to_list(x))).alias('variance_amount')

      ).select(pl.exclude('amount','balance'))





      # x_dataframe.with_columns(
      #                         pl.col("names").map_elements(lambda x: 1 if x.startswith('M') else 0 ).alias("commercial"),
      #                         pl.col("names").map_elements(lambda x: compute_mean_var(self.raw_data, x) ).alias("mean"),
      #                         pl.col("names").map_elements(lambda x: compute_mean_var(self.raw_data, x, var=True) ).alias("var")
      #                         )

      self.raw_data =  self.raw_data.with_columns(pl.col("nameOrig").map_elements(lambda x: node_names[x]).alias("nameOrig"))
      self.raw_data =  self.raw_data.with_columns(pl.col("nameDest").map_elements(lambda x: node_names[x]).alias("nameDest"))
      self.raw_data =  self.raw_data.with_columns(pl.col("step").map_elements(lambda x: x%24).alias("step"))
      x = torch.tensor(group4.select(pl.exclude('names')).to_numpy(), dtype=torch.float)


      #breakpoint()
      chtype1 = self.raw_data.with_columns(pl.col('step').cast(str).alias("step"), pl.col('type').cast(str).alias("type"), pl.col('amount').cast(str).alias("amount"))
      red1 = chtype1.select(pl.reduce(lambda x, y: x + y, exprs=pl.col('step','amount') ).alias('edge_attr'), pl.col("type"))
      red2 = red1.select(pl.reduce(lambda x, y: x + y, exprs=pl.col('type','edge_attr') ).alias('edge_attr'))
      chtype2 = red2.with_columns(pl.col('edge_attr').cast(float).alias('edge_attr'))
      edge_attr = torch.tensor(chtype2.to_numpy(), dtype=torch.float)
      y = torch.tensor(self.raw_data.select(pl.col('isFraud')).to_numpy(), dtype=torch.float)

      edge_index = torch.tensor(self.raw_data.select(pl.col('nameOrig', 'nameDest')).to_numpy(), dtype=torch.int).t()
      self.raw_data = None

      json_data = {
          "edge_attr": edge_attr.tolist(),
          "y": y.tolist(),
          "edge_index": edge_index.tolist(),
          "x": x.tolist()
      }

      with open('graph_data.json', 'w') as f:
        json.dump(json_data,f)



      #breakpoint()
      data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)
      data.y = y

      #self.split = T.RandomNodeSplit(num_val=val_prc, num_test=test_prc)
      self.data = [data]


    def __getitem__(self, index):
      return self.data[0]

    def __len__(self):
      return 1


    def collate(self, data):

      #breakpoint()
      return data[0],self.train_edges, self.val_edges, self.test_edges



    def get_dataloader(self):
      return DataLoader(self, batch_size=1, shuffle=False, collate_fn = self.collate)


    def to_device(self):
      #breakpoint()
      self.data[0].x.to(DEVICE)
      self.data[0].y.to(DEVICE)
      self.data[0].edge_attr.to(DEVICE)
      self.data[0].edge_index.to(DEVICE)
      self.data[0].train_mask.to(DEVICE)
      self.data[0].val_mask.to(DEVICE)
      self.data[0].test_mask.to(DEVICE)









In [ ]:
dataset = FraudDetectionDataset("PS_20174392719_1491204439457_log.csv")


In [ ]:
dataset.raw_data.filter((abs(pl.col('oldbalanceOrg') - pl.col('newbalanceOrig')) != pl.col('amount')) | (abs(pl.col('oldbalanceDest') - pl.col('newbalanceDest')) != pl.col('amount')))

# dataset.raw_data.filter(abs(pl.col('oldbalanceOrg') - pl.col('newbalanceOrig')) != pl.col('amount') |
#             abs(pl.col('oldbalanceDest') - pl.col('newbalanceDest')) != pl.col('amount') | pl.col('amount') == 0 | pl.col('nameOrig').is_in([pl.col('nameDest')])
#         )

In [ ]:
inconsistent_data = dataset.extract_inconsistent_transactions()
d = compute_kind_inconsistence(inconsistent_data)
plot_histogram(d)


In [ ]:
print(dataset.raw_data.columns)
dataset.remove_inconsistent()
dataset.convert_type()
dataset.build_graph(0.1,0.2)


['step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg', 'newbalanceOrig', 'nameDest', 'oldbalanceDest', 'newbalanceDest', 'isFraud', 'isFlaggedFraud']


<ipython-input-62-3577633b5231>:85: PolarsInefficientMapWarning: 
Expr.map_elements is significantly slower than the native expressions API.
Only use if you absolutely CANNOT implement your logic otherwise.
Replace this expression...
  - pl.col("nameOrig").map_elements(lambda x: ...)
with this one instead:
  + pl.col("nameOrig").replace(node_names)

  self.raw_data =  self.raw_data.with_columns(pl.col("nameOrig").map_elements(lambda x: node_names[x]).alias("nameOrig"))
<ipython-input-62-3577633b5231>:86: PolarsInefficientMapWarning: 
Expr.map_elements is significantly slower than the native expressions API.
Only use if you absolutely CANNOT implement your logic otherwise.
Replace this expression...
  - pl.col("nameDest").map_elements(lambda x: ...)
with this one instead:
  + pl.col("nameDest").replace(node_names)

  self.raw_data =  self.raw_data.with_columns(pl.col("nameDest").map_elements(lambda x: node_names[x]).alias("nameDest"))
<ipython-input-62-3577633b5231>:87: PolarsInefficien

> <ipython-input-62-3577633b5231>(92)build_graph()
     90 
     91       breakpoint()
---> 92       chtype1 = self.raw_data.with_columns(pl.col('step').cast(str).alias("step"), pl.col('type').cast(str).alias("type"), pl.col('amount').cast(str).alias("amount"))
     93       red1 = chtype1.select(pl.reduce(lambda x, y: x + y, exprs=pl.col('step','amount') ).alias('edge_attr'), pl.col("type"))
     94       red2 = red1.select(pl.reduce(lambda x, y: x + y, exprs=pl.col('type','edge_attr') ).alias('edge_attr'))

ipdb> n
> <ipython-input-62-3577633b5231>(93)build_graph()
     91       breakpoint()
     92       chtype1 = self.raw_data.with_columns(pl.col('step').cast(str).alias("step"), pl.col('type').cast(str).alias("type"), pl.col('amount').cast(str).alias("amount"))
---> 93       red1 = chtype1.select(pl.reduce(lambda x, y: x + y, exprs=pl.col('step','amount') ).alias('edge_attr'), pl.col("type"))
     94       red2 = red1.select(pl.reduce(lambda x, y: x + y, exprs=pl.col('type','edge_a

In [ ]:
df = pl.DataFrame(
    {
        "a": ["[1]", "[2]", "[3]"],
        "b": ["[0]", "[1]","[2]"],
    }
)

df2 = pl.DataFrame(
    {
        "a": [1, 2, 3],
        "b": [0, 1, 2],
    }
)
def concatenate(acc, x):
  for e in x:
    acc.append(e)
  return acc

df = df.with_columns(
    pl.reduce(function=lambda acc, x: acc +"|"+ x, exprs=pl.col("*")).alias("sum")
)

df2 = df2.with_columns(
    pl.reduce(function=lambda acc, x: acc + x, exprs=pl.col("*")).alias("sum")
)

print(df)
print(df2)


shape: (3, 3)
┌─────┬─────┬─────────┐
│ a   ┆ b   ┆ sum     │
│ --- ┆ --- ┆ ---     │
│ str ┆ str ┆ str     │
╞═════╪═════╪═════════╡
│ [1] ┆ [0] ┆ [1]|[0] │
│ [2] ┆ [1] ┆ [2]|[1] │
│ [3] ┆ [2] ┆ [3]|[2] │
└─────┴─────┴─────────┘
shape: (3, 3)
┌─────┬─────┬─────┐
│ a   ┆ b   ┆ sum │
│ --- ┆ --- ┆ --- │
│ i64 ┆ i64 ┆ i64 │
╞═════╪═════╪═════╡
│ 1   ┆ 0   ┆ 1   │
│ 2   ┆ 1   ┆ 3   │
│ 3   ┆ 2   ┆ 5   │
└─────┴─────┴─────┘


In [ ]:
def convert_to_networkx(graph):
	g = to_networkx(graph, node_attrs=['x'])

	return g

def plot_graph(g):
	plt.figure(figsize=(9,7))
	nx.draw_spring(g, node_size=30, arrows=False)
	plt.show()



g = convert_to_networkx(dataset.data[0])
plot_graph(g)
del g


In [ ]:
class FraudDetectionModule(L.LightningModule):

  def __init__(self,nodes_size, hidden_layer_size, depth, lr=1e-3, wd=5e-4):
    super().__init__()
    layers = [GCNConv(nodes_size, hidden_layer_size), nn.ReLU()]
    for _ in range(depth-2):
      layers.append(GCNConv(hidden_layer_size, hidden_layer_size))
      layers.append(nn.ReLU())
    self.gnn = Sequential(*layers)
    self.classifier = nn.Linear(hidden_layer_size, 1)
    self.sigmoid = nn.Sigmoid()
    self.lr = lr
    self.wd = wd
    self.validation_step_outputs = []
    self.f1 = BinaryF1Score()

    self.loss = nn.BCEWithLogitsLoss()


  def forward(self,data):
    edge_index = data.edge_index
    edge_attr = data.edge_attr
    x = data.x
    #breakpoint()

    self.gnn(x, edge_index, edge_attr)



    return self.sigmoid(self.classifier(x))



  def training_step(self, batch, batch_idx):
    #breakpoint()

    #print("training")

    z = self.forward(batch)
    loss = self.loss(z[batch.train_mask],batch.y[batch.train_mask])
    # Logging to TensorBoard (if installed) by default
    self.log("train_loss", loss, prog_bar=True)
    return loss


  def validation_step(self, batch, batch_idx):
    #breakpoint()

    #print("validation")

    z = self.forward(batch)
    print(z.shape)
    val_loss = self.loss(z[batch.val_mask],batch.y[batch.val_mask])
    f1 = self.f1(z[batch.val_mask],batch.y[batch.val_mask])
    #wndb.log({"val_loss": val_loss,"f1-score":f1})
    self.log_dict({"val_loss": val_loss,"f1-score":f1}, prog_bar=True)


  def configure_optimizers(self):
    optimizer = optim.Adam(self.parameters(), lr=self.lr, weight_decay=self.wd)
    return optimizer




In [ ]:


# wndb.init(
#     project="datamining-hw4",

#     # track hyperparameters and run metadata
#     config={
#     "learning_rate": LR ,
#     "weight decay": WD ,
#     "layers": DEPTH ,
#     "hidden layers size": H_SIZE
#     })
#dataset.to_device()
loader = dataset.get_dataloader()
#breakpoint()

model = FraudDetectionModule(dataset.data[0].num_node_features,dataset.data[0].edge_attr.shape[1] ,H_SIZE,DEPTH,LR,WD)
#model.to(DEVICE)

trainer = L.Trainer(deterministic=True,max_epochs=15,accelerator=ACCELERATOR,precision="16-mixed")



INFO: Using 16bit Automatic Mixed Precision (AMP)
INFO:lightning.pytorch.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
# for name, param in model.named_parameters():
    # print(f"Parameter {name} device:", param.device)

trainer.fit(model, loader, loader)
#wndb.finish()

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name       | Type             | Params
------------------------------------------------
0 | classifier | Linear           | 18    
1 | relu       | ReLU             | 0     
2 | sigmoid    | Sigmoid          | 0     
3 | f1         | BinaryF1Score    | 0     
4 | loss       | CrossEntropyLoss | 0     
------------------------------------------------
18        Trainable params
0         Non-trainable params
18        Total params
0.000     Total estimated model params size (MB)
INFO:lightning.pytorch.callbacks.model_summary:
  | Name       | Type             | Params
------------------------------------------------
0 | classifier | Linear           | 18    
1 | relu       | ReLU             | 0     
2 | sigmoid    | Sigmoid          | 0     
3 | f1         | BinaryF1Score    | 0     
4 | loss       | CrossEntropyLoss | 0     
---------------------

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.


torch.Size([9073900, 2])


IndexError: ignored

In [ ]:
dataset.analize_data(find_empty_records=False)


# Calcola la percentuale di record fraudolenti in inconsistent_data
print("inconsistent fraud")
print(inconsistent_data.filter(pl.col('isFraud') == 1).height / inconsistent_data.height)

# Calcola la percentuale di record non fraudolenti in inconsistent_data
print("inconsistent not fraud")
print(inconsistent_data.filter(pl.col('isFraud') == 0).height / inconsistent_data.height)

# Calcola la percentuale di record con oldbalanceOrg - newbalanceOrig > oldbalanceDest - newbalanceDest
print("given > gotten")
print(dataset.raw_data.filter(abs(pl.col('oldbalanceOrg') - pl.col('newbalanceOrig')) > abs(pl.col('oldbalanceDest') - pl.col('newbalanceDest'))).height / inconsistent_data.height)

# Calcola la percentuale di record con oldbalanceOrg - newbalanceOrig < oldbalanceDest - newbalanceDest
print("given < gotten")
print(dataset.raw_data.filter(abs(pl.col('oldbalanceOrg') - pl.col('newbalanceOrig')) < abs(pl.col('oldbalanceDest') - pl.col('newbalanceDest'))).height / inconsistent_data.height)

# Calcola il numero di valori distinti nella colonna 'nameOrig'
print("distinct names")
print( pl.DataFrame({"names": pl.concat([dataset.raw_data['nameDest'] , dataset.raw_data['nameOrig']])}).select("names").unique().height)


# Calcola la percentuale di record con isFlaggedFraud e isFraud contemporaneamente
print("isFlaggedFraud and isFraud")
print(dataset.raw_data.filter((pl.col('isFlaggedFraud') == 1) & (pl.col('isFraud') == 1)).height / dataset.raw_data.filter(pl.col('isFraud') == 1).height)

# Plot delle categorie
plot_categories(dataset.raw_data)


----HEAD----
shape: (5, 11)
┌──────┬──────────┬──────────┬─────────────┬───┬────────────┬────────────┬─────────┬───────────────┐
│ step ┆ type     ┆ amount   ┆ nameOrig    ┆ … ┆ oldbalance ┆ newbalance ┆ isFraud ┆ isFlaggedFrau │
│ ---  ┆ ---      ┆ ---      ┆ ---         ┆   ┆ Dest       ┆ Dest       ┆ ---     ┆ d             │
│ i64  ┆ str      ┆ f64      ┆ str         ┆   ┆ ---        ┆ ---        ┆ i64     ┆ ---           │
│      ┆          ┆          ┆             ┆   ┆ f64        ┆ f64        ┆         ┆ i64           │
╞══════╪══════════╪══════════╪═════════════╪═══╪════════════╪════════════╪═════════╪═══════════════╡
│ 1    ┆ PAYMENT  ┆ 9839.64  ┆ C1231006815 ┆ … ┆ 0.0        ┆ 0.0        ┆ 0       ┆ 0             │
│ 1    ┆ PAYMENT  ┆ 1864.28  ┆ C1666544295 ┆ … ┆ 0.0        ┆ 0.0        ┆ 0       ┆ 0             │
│ 1    ┆ TRANSFER ┆ 181.0    ┆ C1305486145 ┆ … ┆ 0.0        ┆ 0.0        ┆ 1       ┆ 0             │
│ 1    ┆ CASH_OUT ┆ 181.0    ┆ C840083671  ┆ … ┆ 21182.0    ┆ 0